### 0. 웹크롤링 (웨이팅)

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from bs4 import BeautifulSoup
import pandas as pd

# 페이지 로드
url = 'https://app.catchtable.co.kr/ct/map/COMMON?showTabs=true&serviceType=WAITING&isInitSearch=0&bottomSheetHeightType=FULL_LIST&isShowListLabelExpanded=1&zoomLevel=10&isNewSearchInMap=1&isShowMapSearchButton=0&centerBoundsLat=37.23713118204578&centerBoundsLng=126.9765592842115&location=CAT011_CAT011001_CAT011002_CAT011003_CAT011004_CAT011005_CAT011006_CAT011007_CAT011008_CAT011009_CAT011010_CAT011_CAT011001_CAT011002_CAT011003_CAT011004_CAT011005_CAT011006_CAT011007_CAT011008_CAT011009_CAT011010&isSearchedInMap=0&uniqueListId=1719466746787'
driver = webdriver.Chrome()
driver.get(url)
time.sleep(5)

data = []

# 모달 창의 스크롤 가능한 영역을 선택
modal_scroll_element = driver.find_element(By.CSS_SELECTOR, '#root > div > div > div > div._53lcbm0._53lcbm1 > div')

# 스크롤하여 모든 데이터를 가져오기
previous_height = driver.execute_script("return arguments[0].scrollHeight", modal_scroll_element)
while True:
    # 페이지 소스 가져오기
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    items = soup.select('.restaurant-info')

    for item in items:
        if item is not None:
            name = item.select_one("h4.name").text.strip() if item.select_one("h4.name") else 'NA'
            cat_loc = item.select_one("span.tags").text.strip() if item.select_one("span.tags") else 'NA'     
            rating = item.select_one("span._1xnbt316._1ltqxco1d").text.strip() if item.select_one("span._1xnbt316._1ltqxco1d") else 'NA'
            review = item.select_one("span._1xnbt317._1ltqxco1g").text.strip() if item.select_one("span._1xnbt317._1ltqxco1g") else 'NA'
            description = item.select_one("p.excerpt").text.strip() if item.select_one("p.excerpt") else 'NA'
            price_range = item.select_one("span.price").text.strip() if item.select_one("span.price") else 'NA'

            data.append([name, cat_loc, rating, review, description, price_range])
            print(name, cat_loc)

        else:
            data.append(['NA', 'NA', 'NA', 'NA', 'NA', 'NA'])

    # 데이터 개수 출력
    print(f"현재까지 수집된 데이터 개수: {len(data)}")

    # 모달 창 내부를 스크롤
    driver.execute_script('arguments[0].scrollTop = arguments[0].scrollHeight', modal_scroll_element)
    time.sleep(3)  # 페이지가 로드될 시간을 줍니다

    # 새로운 높이를 확인하고 이전 높이와 비교
    new_height = driver.execute_script("return arguments[0].scrollHeight", modal_scroll_element)
    if new_height == previous_height:
        break  # 더 이상 새로운 데이터가 로드되지 않으면 루프 종료
    previous_height = new_height

driver.quit()

# 데이터프레임으로 변환
columns = ['Name', 'Cat_loc', 'Rating', 'Review', 'Description', 'Price_Range']
df = pd.DataFrame(data, columns=columns)

# CSV 파일로 저장
df.to_csv('catchtable_waiting_data3.csv', index=False)

# 데이터 출력
print(df.head())

### 1. 전처리 

##### Location_1 수정 

In [ ]:
# 데이터 불러오기 
df=pd.read_csv('combined_review_reservation_data.csv')
df.head()

In [ ]:
# 위치 데이터 데이팅 및 수정 
# << Location_1 >>

# 삭제 데이터: 
# 1. 글렌알라키 인 서울 : 삭제. 4월에 일시적으로 진행된 팝업행사였음 
# 2. df['Name']=='난로 인사이트 2024' ==> 식당이 아닌 것 같아 삭제 대상. 강연 같음 
df.drop(df[df['Name']=='글렌알라키 인 서울'].index, inplace=True)
df.drop(df[df['Name']=='난로 인사이트 2024'].index, inplace=True)

#  Location Null값인 하기 데이터 아래와 같이 수정 (구 기준으로 먼저, 이후 동 기준 데이터 추가)
# df['Name']=='솔솥 경의선숲길점' ==> 마포구 염리동
# df['Name']=='토리키치 압구정로데오점' ==> 강남구 신사동
# df['Name']=='숙성주당' ==> 영등포 문래동
# df['Name']=='상록회관 연탄구이 신림역점' ==> 관악구 신림동
df.loc[df['Name'] == '솔솥 경의선숲길점', 'Location'] = '마포'
df.loc[df['Name'] == '토리키치 압구정로데오점', 'Location'] = '강남'
df.loc[df['Name'] == '숙성주당', 'Location'] = '영등포'
df.loc[df['Name'] == '상록회관 연탄구이 신림역점', 'Location'] = '관악'

In [ ]:
# 위치 카테고리 대분류 정리 (Location_1 열 생성하여 추가할 내용)
location_c={
    '강남구': ['도산공원','압구정로데오','강남','신사','삼성','청담','가로수길','역삼','압구정','논현','대치','개포','도곡','선릉','매봉','강남구청','선정릉',
                '삼성동', '일원','수서','가로수','개포동'],
    '강동구': ['성내','천호','명일','암사','길동','고덕','둔촌동','상일'],
    '강북구': ['수유','강북','미아','우이동'],
    '강서구': ['마곡', '등촌','방화','강서','장승배기','발산','등촌동','김포공항'],
    '관악구': ['서울대입구','샤로수길','신림','관악','봉천','서울대학교','낙성대'],
    '광진구': ['군자','건대','광장동','구의','중곡','자양','뚝섬'],
    '구로구': ['구로','신도림','구로디지털단지','고척','개봉'],
    '금천구': ['금천','가산디지털단지','독산', '가산','시흥'],
    '노원구': ['공릉','하계','상계','월계','중계','노원'],
    '도봉구': ['창동','쌍문','도봉'],
    '동대문구': ['청량리','회기','장안','이문','신설동','동대문','전농','답십리'],
    '동작구': ['사당','노량진','상도','보라매','신대방','이수','동작','흑석'],
    '마포구': ['합정','연남','상수','망원','마포','홍대','공덕','상암','이대','서교','도화','연','신수'],
    '서대문구': ['신촌','연희','충정로','서대문','독립문'],
    '서초구': ['반포','서초','방배','교대','양재','서래마을','잠원', '반포동','솔빛섬','청계산'],
    '성동구': ['서울숲','성수','왕십리','옥수','상왕십리','금호','하왕십리', '행당','마장','서울'],
    '성북구': ['성신여대','보문','하월곡','정릉','동소문','성북','안암','동선','장위', '고려대','석관'],
    '송파구': ['잠실','석촌','송파','문정','방이','가락','잠실새내','송리단길', '올림픽공원'],
    '양천구': ['목동','오금','신정'],
    '영등포구': ['영등포','여의도','문래','신길','선유도','당산','양평'],
    '용산구': ['삼각지','서울역','남영','신용산','한남','용산','이태원','해방촌','이촌','후암','경리단길'],
    '은평구': ['은평','연신내','응암','녹번'],
    '종로구': ['안국','삼청','혜화','광화문','북촌','종로','서촌','익선','종로3가','인사동','종각','대학로', '와룡','평창동'],
    '중구': ['을지로', '약수', '회현', '명동','장충동','충무로','신당', '시청', '충무'],
    '중랑구':['중랑','면목','중화']}

In [ ]:
# location 값을 location_c를 참고하여 location_1 열에 추가 :
# 1. 사용자 함수
def location_list(location):
    for key, values in location_c.items():
        if any(value in location for value in values):
            return key
    return ''

df['Location_1']=df['Location'].apply(location_list)
df

In [ ]:
# "서울역" location_1 수정 
df.loc[df['Location']=='서울역', 'Location_1']='용산구'

# "연신내" location_1 수정
df.loc[df['Location']=='연신내', 'Location_1']='은평구' 

##### Location_2 생성/에디팅

In [ ]:
# 주요 9개 지역구의 세부 지역 디테일 정리를 위해 column 추가 (기존 location과 동일하게 데이터 복사한 후 에디팅 진행)
df['Location_2']=df['Location']

In [ ]:
# << Location_2 >>
# 주요 9개 지역구의 세부 지역 디테일 확인을 위한 Location 데이터 에디팅:
df.loc[df['Location']=='충무', 'Location_2']='충무로'
df.loc[df['Location']=='개포동', 'Location_2']='개포'
df.loc[df['Location']=='삼성동', 'Location_2']='삼성'
df.loc[df['Location']=='서울', 'Location_2']='성수'
df.loc[df['Location']=='대학로', 'Location_2']='혜화'
df.loc[df['Location']=='신길', 'Location_2']='영등포'
df.loc[df['Location']=='올림픽공원', 'Location_2']='방이'
df.loc[df['Location']=='동작', 'Location_2']='노량진'
df.loc[df['Location']=='이촌', 'Location_2']='신용산/이촌'
df.loc[df['Location']=='신용산', 'Location_2']='신용산/이촌'
df.loc[df['Location']=='연', 'Location_2']='연남'
df.loc[df['Location']=='가로수', 'Location_2']='가로수길'
df.loc[df['Location']=='솔빛섬', 'Location_2']='반포'
df.loc[df['Location']=='서교', 'Location_2']='홍대'

# 강남구 추가 수정: 
df.loc[df['Location']=='선정릉', 'Location_2']='선릉/선정릉'
df.loc[df['Location']=='선릉', 'Location_2']='선릉/선정릉'
df.loc[df['Location']=='개포', 'Location_2']='도곡'
df.loc[df['Location']=='개포동', 'Location_2']='기타'
df.loc[df['Location']=='수서', 'Location_2']='기타'

# 마포구 추가 수정 : 
df.loc[df['Location']=='상왕십리', 'Location_2']='왕십리/상왕십리'
df.loc[df['Location']=='왕십리', 'Location_2']='왕십리/상왕십리'

# 송파구 추가 수정:
df.loc[df['Location']=='송리단길', 'Location_2']='석촌'

##### 3. 음식 카테고리 에디팅

In [ ]:
# 음식 카테고리 불량 데이터 에디팅 
# Category 이상 데이터 하기와 같이 수정
df.loc[df['Category']=='장안', 'Category']='육류'
df.loc[df['Category']=='여의도', 'Category']='육류' 
df.loc[df['Category']=='이태원', 'Category']='와인바' 
df.loc[df['Category']=='노원', 'Category']='이자카야' 
df.loc[df['Category']=='역삼', 'Category']='육류'
df.loc[df['Category']=='성신여대', 'Category']='이자카야' 
df.loc[df['Category']=='신사', 'Category']='한식'
df.loc[df['Category']=='연남', 'Category']='위스키바'
df.loc[df['Category']=='용산', 'Category']='위스키바'
df.loc[df['Category']=='신림', 'Category']='육류' 
df.loc[df['Category']=='마포', 'Category']='한식' 
df.loc[df['Name']=='토리키치 압구정로데오점','Category']='이자카야'
df.loc[df['Name']=='청국장과보리밥 수서점','Category']='한식' 
df.loc[df['Name']=='꼬따 cotta','Category']='이탈리아 음식' 
df.loc[df['Name']=='선수촌가든','Category']='육류' 
df.loc[df['Name']=='레인보우포차 동대문점','Category']='요리주점'
df.loc[df['Name']=='진가와 여의도점','Category']='일식'
df.loc[df['Name']=='진가와 광화문점','Category']='일식' 
df.loc[df['Name']=='강남쪽갈비 논현역점','Category']='육류'
df.loc[df['Name']=='프렌밀리','Category']='와인바'
df.loc[df['Name']=='재즈라운지','Category']='와인바'
df.loc[df['Name']=='플랫다이닝카페','Category']='브런치'
df.loc[df['Name']=='강호연파 성수 숲속산장점','Category']='샤브샤브' 
df.loc[df['Name']=='올나잇','Category']='맥주,호프'
df.loc[df['Name']=='토리키치 압구정로데오점','Category']='이자카야'
df.loc[df['Name']=='토리키치 청담점','Category']='이자카야'
df.loc[df['Name']=='야키토리 이크','Category']='이자카야' 
df.loc[df['Name']=='키친485','Category']='이탈리아 음식'
df.loc[df['Name']=='신사만장어','Category']='장어 전문' 
df.loc[df['Name']=='블루메쯔 노들섬','Category']='유러피안음식' 
df.loc[df['Name']=='카키바움','Category']='유러피안음식' 
df.loc[df['Name']=='Rirr','Category']='이탈리아 음식' 
df.drop(df[df['Name']=='가히 쿠킹클래스'].index, inplace=True) # 삭제 대상. 체험 액티비티임 
df.drop(df[df['Name']=='알로소 윈터리빙룸'].index, inplace=True) # 삭제 대상. 정식 식당이 아닌 이벤트성 팝업으로 보임

### 2. 분석 및 시각화

In [ ]:
# 주요 9개 지역 별로 세부 지역 밀도 수정 후 재확인:
# 식당 밀도 높은 지역 중 세부 지역만 따로 빼기
high_density_locations=['강남구','마포구','용산구','성동구','종로구','송파구','중구','영등포구','서초구']
df_hdl=df[df['Location_1'].isin(high_density_locations)]
df_hdl

In [ ]:
# 주요 9개 지역 개별로 세부 지역 변수화 + 시각화
# high_density_locations=['강남구','마포구','용산구','성동구','종로구','송파구','중구','영등포구','서초구']
hdl_gangnam=df[df['Location_1']=='강남구']
hdl_mapo=df[df['Location_1']=='마포구']
hdl_yongsan=df[df['Location_1']=='용산구']
hdl_sungdong=df[df['Location_1']=='성동구']
hdl_jongro=df[df['Location_1']=='종로구']
hdl_songpa=df[df['Location_1']=='송파구']
hdl_jonggu=df[df['Location_1']=='중구']
hdl_youngdeungpo=df[df['Location_1']=='영등포구']
hdl_seocho=df[df['Location_1']=='서초구']

##### 주요 9개 지역별 식당 별점

In [ ]:
# 주요 9개 지역별 식당 별점 분포 (박스플롯)
df_hdl['Location_1'].unique()

# fig, axes=plt.subplots(2,5, figsize=(10,8))
# high_density_locations=['강남구','마포구','용산구','성동구','종로구','송파구','중구','영등포구','서초구']
order=['강남구','마포구','용산구','성동구','종로구','송파구','중구','영등포구','서초구']

plt.figure(figsize=(16,8))
plt.title('주요 9개 지역별 식당 별점', fontsize=18)

sns.boxplot(x='Location_1', y='Rating', data=df_hdl, order=order, palette='cool')
plt.ylabel('별점', fontsize=14)
plt.xlabel('지역구', fontsize=14)

plt.tight_layout()
plt.show()

In [ ]:
# 주요 9개 지역 리뷰수 평균
print(df_hdl['Review'].mean().round(1))

##### 주요 9개 지역별 식당 평균 리뷰 수

In [ ]:
# 주요 9개 지역별 식당 평균 리뷰 수 (표)
df_hdl_average_review=df_hdl.groupby('Location_1')['Review'].mean().round(1).reset_index(name='count')
df_hdl_average_review.sort_values(by='count', ascending=False)

# 주요 9개 지역별 식당 평균 리뷰 수 (그래프)
df_hdl['Location_1'].unique()

# fig, axes=plt.subplots(2,5, figsize=(10,8))
# high_density_locations=['강남구','마포구','용산구','성동구','종로구','송파구','중구','영등포구','서초구']
order=['강남구','마포구','용산구','성동구','종로구','송파구','중구','영등포구','서초구']

plt.figure(figsize=(16,8))
plt.title('주요 9개 지역별 식당 리뷰 수', fontsize=18)

sns.barplot(x='Location_1', y='count', data=df_hdl_average_review, order=order, palette='cool')
plt.ylabel('평균 리뷰 수', fontsize=14)
plt.xlabel('지역구', fontsize=14)

plt.tight_layout()
plt.show()

##### Top4 지역구 음식 카테고리

In [ ]:
# 주요 지역 4개만 (식당수로 top4 지역)
hdl_rating_varied=['강남구', '마포구', '용산구', '성동구']
df_hdl_top4=df[df['Location_1'].isin(hdl_rating_varied)]
df_hdl_top4.head()

In [ ]:
# top4 지역별 식당의 음식 카테고리 비중 (count로)
df_hdl_top4_piv=df_hdl_top4.groupby(['Location_1','Category_1'])['Category'].count().unstack()
df_hdl_top4_piv=df_hdl_top4_piv[df_hdl_top4_piv.sum().sort_values(ascending=False).index]

location_order=df_hdl_top4_piv.sum(axis=1).sort_values(ascending=False).index
df_hdl_top4_piv=df_hdl_top4_piv.loc[location_order]

df_hdl_top4_piv

In [ ]:
# top4 지역별 식당의 음식 카테고리 비중 (%)
# 데이터프레임 재구성: 각 지역구에서의 음식 카테고리 비율 계산
df_hdl_top4_piv_percent = df_hdl_top4_piv.div(df_hdl_top4_piv.sum(axis=1), axis=0) * 100

# 비율이 0인 경우 NaN이 되므로 결측치를 0으로 채우기
df_hdl_top4_piv_percent = df_hdl_top4_piv_percent.fillna(0)

# 정렬: 지역구 크기에 따라 정렬
location_order = df_hdl_top4_piv.sum(axis=1).sort_values(ascending=False).index
df_hdl_top4_piv_percent = df_hdl_top4_piv_percent.loc[location_order]

# 결과 확인
print(df_hdl_top4_piv_percent.round(1))

In [ ]:
# top4 지역별 식당의 음식 카테고리 비중 (%) 히트맵
df_hdl_top4_piv_filled = df_hdl_top4_piv.fillna(0)

# 그래프 크기 설정
plt.figure(figsize=(12, 8))

# 히트맵 그리기
sns.heatmap(df_hdl_top4_piv_filled, cmap='coolwarm', annot=True, fmt='.0f')

# 제목과 축 라벨 설정
plt.title('Top 4 지역구의 음식 카테고리별 카테고리 개수', fontsize=16)
plt.xlabel('음식 카테고리', fontsize=14)
plt.ylabel('Location_1', fontsize=14)

# 그래프 표시
plt.show()

##### Top4 지역구 양/한/일식 별점

In [ ]:
# 6. 주요 4개 지역 중 
hdl_rating_varied = ['강남구', '마포구', '용산구', '성동구']
food_cat = ['양식', '한식', '일식']

fig, axes = plt.subplots(1, 4, figsize=(20, 5), sharey=True)
plt.suptitle('Top4 지역구의 양식/한식/일식 별점', fontsize=16)

for i, loc in enumerate(hdl_rating_varied):
    subset = df_hdl[(df_hdl['Location_1'] == loc) & (df_hdl['Category_1'].isin(food_cat))]
    sns.boxplot(x='Category_1', y='Rating', data=subset, ax=axes[i], palette='cool')
    axes[i].set_title(loc)
    axes[i].set_xlabel('음식 카테고리')
    axes[i].set_ylabel('별점')

plt.tight_layout()
plt.show()

### 3. 텍스트 분석 

현재 진행중. 완료 후 공유 예정